In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from data.acronymDataset import AcronymDataset

/opt/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=config.max_position_embeddings)
pre_trained_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [33]:
%load_ext autoreload
%autoreload 2
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path, tokenizer=tokenizer)
data = dataset.data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
dataset.preprocss_dataset()

In [35]:
train_loader, val_loader = dataset.get_dataloaders(train_size=0.9, batch_size=16)

In [38]:
batch = next(iter(train_loader))

In [39]:
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [40]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2)
four_labels_head = ClassificationHead(in_features=in_features, out_features=4)

classifiers = [two_labels_head, four_labels_head]


In [41]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [42]:

output = multi_head_model(batch, 0)
output

tensor([[0.4940, 0.4430],
        [0.5081, 0.4410],
        [0.5084, 0.4301],
        [0.5129, 0.4618],
        [0.5210, 0.4448],
        [0.5062, 0.4501],
        [0.5413, 0.4487],
        [0.5049, 0.4565],
        [0.5057, 0.4518],
        [0.5130, 0.4646],
        [0.5001, 0.4312],
        [0.4987, 0.4280],
        [0.5165, 0.4539],
        [0.5189, 0.4585],
        [0.5090, 0.4609],
        [0.5099, 0.4439]], grad_fn=<SigmoidBackward0>)

In [43]:
output.shape

torch.Size([16, 2])

In [9]:
output = multi_head_model(batch, 1)
output

tensor([[0.5332, 0.4287, 0.5225, 0.5521]], grad_fn=<SigmoidBackward0>)